# Stat 140 Lab 1: Introduction to R and Categorical Data

In this lab, you'll practice two tasks that you face when you are thinking about a new data set:

1. reading data in a spreadsheet into R; and
2. doing some basic exploratory analysis (for today, we will focus on categorical variables in this step)

By the end of this lab, I hope you feel like you could do the following tasks (perhaps referring back to this lab for the details -- I don't expect that you will have all of this memorized):
 * read a spreadsheet-like data file into R
 * determine the number of observations and variables in a data set
 * determine the data types of variables in a data set
 * convert character variables to factor variables (either ordered or not, as appropriate)
 * calculate how many observational units fall into each category of a categorical variable (but we will revisit this next class)
 * make bar plots of categorical variables (but we will revisit this next week)

We will work with data from the "National Health and Nutrition Examination Survey", a survey that is conducted every year and records demographic and health information about a large nationally-representative sample of Americans.  We will work with a relatively small subset of this data set; I have selected just a few variables and a subset of the participants from the 2011-2012 version of the survey.

## Reading Data Into R

R comes with some basic default functions that can read in data, but they're not very useful.  Instead, we will use functions in the `readr` package for R.  A **package** is a group of extra functions that don't come with R by default, but can be used to extend R's functionality.  The `readr` package includes functions that can read in a variety of different types of data files.  R packages are stored in a **library**.  We can load the `readr` package, and make the functions it provides for reading data available to use, with the first command in the next cell (go ahead and run these commands now):

In [ ]:
library(readr)
library(dplyr)
library(ggplot2)
library(mosaic)

While we were at it, we also loaded three other R packages that will be useful for this lab.  The `dplyr` package has lots of functionality that's useful for cleaning and transforming data, the `ggplot2` package is what we'll use for making plots, and the `mosaic` package has some functions that are useful for summarizing data.  We'll get to those a little later on.

As we discussed, today's data set is located in a "csv" file (csv stands for comma separated value).  To read in data in a csv file, we can use the `read_csv` function that the `readr` package provides.  We have to tell `read_csv` where it can find the data file we want.  In this case, we can pass in the location on the course website where the file is stored, between two quotation marks.  Go ahead and run the next cell to read in the data:

In [ ]:
nhanes <- read_csv("http://www.evanlray.com/stat140_s2018/lecture/20180125_intro_to_r/nhanes.csv")

The first part of the above line, `nhanes <-`, says "store the output of the `read_csv` function in a variable called `nhanes`".

The data is now ready for us to interact with in R!  To verify that, you can run the following cell, which does three things:

1. Looks at the first few rows of the `nhanes` data frame using the `head` function.  This function is nice because the output has nice formatting.
2. Looks at a more detailed summary of the first few rows of the `nhanes` data frame using the `str` function.  This function gives you a little more information -- in particular, it tells you how many observations and variables are in the data frame, and the data type of each variable in the data frame.
3. Looks at the dimensions (number of rows and columns) of the `nhanes` data frame using the `dim` function.  This information was already available in the output from `str` -- but sometimes it's useful to get just the information you need, without all that extra stuff.

In [ ]:
head(nhanes)
str(nhanes)
dim(nhanes)

Before we get too much further, take a careful look at the output of the `head` command above.  I would like to draw your attention to two important points:

1. There are a bunch of `NA` values in this data set!  In R an `NA` represents missing data.  Some of these missing values "make sense" -- for example, the `PregnantNow` variable is missing for all of the male respondents in the data set.  Also, we don't have any information about the `Education` level or `MaritalStatus` of the first or the fifth respondent -- but notice that thes participants were aged 14 and 5, respectively.  However, if we plan to do an analysis involving variables with missing values, we need to think about **why** those responses are missing, and **whether that missing data affects the validity of our analysis**.  In some cases, it's possible to deal with missing data like this, but we will not discuss those methods much in this class.
2. Let's think carefully about the categorical variables in this data set.  For some of these variables, like `MaritalStatus` and `PregnantNow`, it seems clear that there are distinct categories for that variable and every respondent will fall neatly into exactly one of those categories.  However, for other variables, like `Gender` and `Race`, it is less clear that there are really distinct categories, or that the categories used in this data set provide a full view of those variables.  The people collecting the data have made a decision about the possible values of those variables, and determined that every subject will be assigned to exactly one category value.  People engaged in other data collection projects have made different decisions, for example allowing a larger and more refined set of categories and/or allowing each person to identify with multiple categories.  It is important to recognize that these **data collection decisions affect the sorts of questions that it is possible to ask and answer** with this data set.  Finally, others of these categorical variables, like `HHIncome` and `Diabetes` can be viewed as a coarsening of an underlying continuous variable.  For example, in this data set we are only told a range of values for the household income, not the exact income level.  Again, this affects what we can do with the data.

OK, on with working with the data!

### 1. Based on the R output above, can you answer the following questions?
#### (a) What is the observational unit in this data set? 

(You can type your answer in here -- but since you're not turning this lab in you don't need to)

#### (b) How many observational units are in this data set?

#### (c) How many variables are in this data set?

#### (d) For each of the variables `ID`, `Gender`, `Age`, `Education`, `HHIncome`, and `Poverty`, answer the following questions: What is the data type of that variable in R right now (numeric, integer, character, factor, ordered factor, ...)?  What type of variable is that conceptually (identifier, discrete quantitative, continuous quantitative, ordinal categorical, or nominal categorical)?  Do R and the actual variable type agree?

You should have found that when R read in the data file, it did a pretty good job of detecting the correct data type for quantitative variables, but not so good a job at detecting the correct data type for categorical variables.  Those variables are all currently stored as characters, but they should be factors!  We'll have to convert them.

I'll write code to convert a couple of these categorical variables to factors, and then have you do the rest.  Hopefully, you found that that `Education` was an ordinal categorical variable and `Gender` was a nominal categorical variable (at least, as it's treated in this data set).  In order to set `Education` to be an ordinal categorical variable, we need to know what all the possible values of that variable in this data set are so that we can set the order correctly.  The `Education` level is one of the following: "8th Grade", "9-11th Grade", "High School", "Some College", or "College Grad".  We can now convert these variables to factors as follows:

In [ ]:
nhanes <- mutate(nhanes,
    Education = factor(Education, levels = c("8th Grade", "9 - 11th Grade", "High School", "Some College", "College Grad"), ordered = TRUE),
    Gender = factor(Gender, levels = c("female", "male"))
    )

In this R cell, we used the `mutate` function, which modifies a data frame by either adding new variables or modifying existing variables.  The `mutate` function comes from the `dplyr` library, which you may recall we loaded above.  We will look at the `mutate` function in more detail in a future lab.  Briefly, here's how it works:

`<name of modified data frame> <- mutate(<original data frame>,
    <new/modified variable 1> = <how to calculate new/modified variable 1>,
    <new/modified variable 2> = <how to calculate new/modified variable 2>,
    ...
    )`

In this case, the data frame that we want to modify is `nhanes`, so that's the first argument to the `mutate` function.

The first variable we want to modify is the `Education` variable.  We calculate the modified `Education` variable by converting it to a `factor` with the specified `levels` and an `ordered = TRUE` argument to say this is an ordinal variable.

The second variable we want to modify is the `Gender` variable.  We calculate the modified `Gender` variable by converting it to a `factor` with the specified `levels`.

We can verify that all of this worked as it was supposed to by looking at the output of another call to `str`: did the variable types update correctly?

In [ ]:
str(nhanes)

### 2. OK, your turn!  There are five categorical variables left in the data set that we need to convert to factors:

 1. `Race`, with levels "Asian", "Black", "Hispanic", "Mexican", "White", "Other"
 2. `MaritalStatus`, with levels "Married", "Widowed", "Divorced", "Separated", "NeverMarried", "LivePartner"
 3. `HHIncome`, with levels "0-4999", "5000-9,999", "10000-14999", "15000-19999", "20000-24999", "25000-34999", "35000-44999", "45000-54999", "55000-64999", "65000-74999", "75000-99999", "more 99999"
 4. `Diabetes`, with levels "No", "Yes"
 5. `PregnantNow`, with levels "No", "Yes", "Unknown"

I've provided starter code for you in the cell below.  You need to fill in an appropriate call to factor() for each variable, then run the code in that cell.  Check to make sure it worked!

In [ ]:
nhanes <- mutate(nhanes,
    Race = ,
    MaritalStatus = ,
    HHIncome = ,
    Diabetes = ,
    PregnantNow = )

str(nhanes)

Congratulations!  We've now finished reading the data in and setting the data types appropriately, so we can move on to actually looking at the data!

## Exploratory Analysis for Categorical Variables

In this lab, we will look at two exploratory tasks for categorical variables:

1. Obtaining counts for how many observational units were in each category of the variable.
2. Making bar plots to display these counts.

We'll start off thinking about just one variable at a time, and then move on to thinking about relationships between two variables.  We will look at quantitative variables next week!

### One variable at a time

Let's think about the `Education` levels of Americans.  How many of the people in our sample have an 8th grade education level or less?  How many of the people in our sample are college graduates?  We can answer these questions with the `tally` function, which is from the `mosaic` package that we loaded earlier:

In [ ]:
tally(~ Education, data = nhanes)

The first argument of this function might look a little funny.  It's a `formula`.  In R, a formula is usually used as a way of specifying a relationship among multiple variables.  We'll see examples of that later in the class.  For now, there's only one variable involved in our calculation; formulas with only one variable will always look like `~ <variable name>`.

There were 212 people in this sample with an 8th grade education or less, and 1128 college graduates.

Notice that there were 1416 people in the sample with no education level recorded!  That's over a quarter of the sample!  It turns out that the education level was only recorded for survey respondents who were 20 years old or older.  The `Education` variable is missing for everyone who was 19 or younger.

We can `filter` the data to look at only the responses for people who were at least 20 with the following commands.  The `filter` function is in the `dplyr` package; we'll look at this function in more detail next week.

In [ ]:
nhanes_20plus <- filter(nhanes, Age >= 20)
tally(~ Education, data = nhanes_20plus)

We'll talk about `filter`ing data more next week, but briefly: the first argument to `filter` is the name of the original data frame we want to filter, and the second argument is a condition specifying how to choose which observational units to keep.

There were only 3 missing values for the `Education` level among adults -- much better!

Remember that the whole point of examining data from a sample is to learn about the population.  Since the NHANES sample was taken carefully and is representative of the U.S. population, the proportion of adults age 20 or more in the sample who are college graduates should be similar to the proportion of adults age 20 or more in the population who are college graduates.  We will say much more about making this sort of inference from the sample to the population in the rest of the class.  For now, let's calculate what proportion of adults in our sample are college graduates.  To do that, we need to find out how many people are left in our filtered data set using the `dim` function:

In [ ]:
dim(nhanes_20plus)

So, our sample included 3587 adults age 20 or more, and of those, 1128 were college graduates.  That means that the proportion of adults in the sample who are college graduates can be calculated as follows:

In [ ]:
1128 / 3587

OK, let's make a plot!  Here is code to make a bar plot showing the counts of people in each level of the `Education` variable in this sample:

In [ ]:
ggplot(data = nhanes, mapping = aes(x = Education)) +
  geom_bar()

We will talk much more about how this code for plotting works next week, so don't worry too much about it!  For today, just notice 4 things:
 * We are using the `ggplot` function, which is provided by the `ggplot2` package that we loaded at the beginning of this document.
 * The first argument of the `ggplot` function is the name of the data frame with the variable we are plotting.
 * The next argument says that we want to plot the `Education` variable, and that variable will go on the `x` axis of the plot
 * On the next line, we are saying that the `geom`etric objects in the plot will be `bar`s.

Again, we'll talk about plotting in much more depth next week.

### 3. Your Turn!

Pick one of the categorical variables in the `nhanes` data frame that seems most interesting to you.  Then,

#### (a) Use the `tally` function to count how many of the observational units in this sample fell into each level of that categorical variable.  If you like, you can also figure out what proportion of the sample fell into each level of that variable.  To start, I recommend copy/paste-ing the code above, and modifying the variable names.  Ask a neighbor or me if you run into trouble!

In [ ]:
# Your code goes here

#### (b) Use `ggplot` to make a bar plot showing how many of the observational units in this sample fell into each level of that categorical variable.  Again, I recommend copy/paste-ing the code above and modifying the variable names.

In [ ]:
# Your code goes here

## Two Categorical Variables

Is there a relationship between `Gender` and `Education` levels in America?  Let's investigate.  Again, we can start by using the `tally` function to get counts, but this time we'll want to get counts for each combination of levels of the `Gender` and `Education` variables.  Here's how we can do that, using the filtered data for adults:

In [ ]:
tally(Education ~ Gender, data = nhanes_20plus)

Suppose we want to compare the proportion of women in our sample who are college graduates to the proportion of men in our sample who are college graduates.  To do that, we need to know the total number of women in the sample and the total number of men in the sample:

In [ ]:
tally( ~ Gender, data = nhanes_20plus)

We can then calculate the proportion of women in the sample who are college graduates as `584/1803` and the proportion of men in the sample who are college graduates as `544/1784`:

In [ ]:
584/1803
544/1784

A slightly higher proportion of women in this sample are college graduates than men.  Is the difference large enough that we can claim that there is a difference in educational attainment for women and men in the population of all adults in America?  We'll look at that sort of question later in the class!

Here are two different ways we can make a bar plot showing the relationship between these two variables.  Note the slight difference in the code to make these plots, and how that shows up in the plot.

In [ ]:
ggplot(data = nhanes, mapping = aes(x = Education, fill = Gender)) +
  geom_bar()

In [ ]:
ggplot(data = nhanes, mapping = aes(x = Gender, fill = Education)) +
  geom_bar()

### 4. Your Turn!

Pick a pair of two categorical variables in the `nhanes` data frame that seem most interesting to you.  Then,

#### (a) Use the `tally` function to count how many of the observational units in this sample fell into each combination of levels of those categorical variables.  If you like, you can also figure out what proportion of the sample fell into each combination of levels of those variables.  To start, I recommend copy/paste-ing the code above, and modifying the variable names.

In [ ]:
# Your code goes here

#### (b) Use ggplot to make a bar plot showing how many of the observational units in this sample fell into each combination of levels of those categorical variables. Again, I recommend copy/paste-ing the code above and modifying the variable names.

In [ ]:
# Your code goes here